In [4]:
import re
import requests
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm

### Skins Scrapping


In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive"
}

weapon_mapping = {
    "cz75-auto": "CZ75-Auto", "desert-eagle": "Desert Eagle", "dual-berettas": "Dual Berettas",
    "five-seven": "Five-SeveN", "glock-18": "Glock-18", "p2000": "P2000", "p250": "P250",
    "r8-revolver": "R8 Revolver", "tec-9": "Tec-9", "usp-s": "USP-S", "zeus-x27": "Zeus x27",
    "mag-7": "MAG-7", "nova": "Nova", "sawed-off": "Sawed-Off", "xm1014": "XM1014",
    "mac-10": "MAC-10", "mp9": "MP9", "mp5-sd": "MP5-SD", "pp-bizon": "PP-Bizon", "p90": "P90", "ump-45": "UMP-45",
    "ak-47": "AK-47", "aug": "AUG", "famas": "FAMAS", "galil-ar": "Galil AR", "m4a1-s": "M4A1-S",
    "m4a4": "M4A4", "sg-553": "SG 553", "awp": "AWP", "g3sg1": "G3SG1", "scar-20": "SCAR-20",
    "ssg-08": "SSG 08", "m249": "M249", "negev": "Negev", "bayonet": "★ Bayonet",
    "bowie-knife": "★ Bowie Knife", "butterfly-knife": "★ Butterfly Knife", "classic-knife": "★ Classic Knife",
    "falchion-knife": "★ Falchion Knife", "flip-knife": "★ Flip Knife", "gut-knife": "★ Gut Knife",
    "huntsman-knife": "Huntsman Knife", "karambit": "Karambit", "kukri-knife": "Kukri Knife",
    "m9-bayonet": "★ M9 Bayonet", "navaja-knife": "★ Navaja Knife", "nomad-knife": "★ Nomad Knife",
    "paracord-knife": "★ Paracord Knife", "shadow-daggers": "★ Shadow Daggers", "skeleton-knife": "★ Skeleton Knife",
    "stiletto-knife": "★ Stiletto Knife", "survival-knife": "★ Survival Knife", "talon-knife": "★ Talon Knife",
    "ursus-knife": "★ Ursus Knife", "bloodhound-gloves": "★ Bloodhound Gloves",
    "broken-fang-gloves": "★ Broken Fang Gloves", "driver-gloves": "★ Driver Gloves", "hand-wraps": "★ Hand Wraps",
    "hydra-gloves": "★ Hydra Gloves", "moto-gloves": "★ Moto Gloves", "specialist-gloves": "★ Specialist Gloves",
    "sport-gloves": "★ Sport Gloves"
}

base_weapon_url = "https://csgoskins.gg/weapons/"

# Use a session to maintain cookies
session = requests.Session()
session.headers.update(headers)

def get_skins_for_weapon(weapon):
    url = f"{base_weapon_url}{weapon}"
    skin_data = []
    max_retries = 5
    delay = 5  # Initial delay in seconds

    while url:
        for attempt in range(max_retries):
            response = session.get(url)
            
            if response.status_code == 200:
                break  # Success, continue scraping
            elif response.status_code == 429:
                wait_time = delay * (2 ** attempt) + random.uniform(0, 2)
                print(f"Rate limited! Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Failed to retrieve {url}. Status Code: {response.status_code}")
                return skin_data
        
        soup = BeautifulSoup(response.text, "html.parser")
        skin_cards = soup.find_all("div", class_="bg-gray-800 rounded-sm shadow-md relative flex flex-wrap h-[545px]")

        for card in skin_cards:
            name_tag = card.find("h2")
            skin_name = name_tag.text.strip() if name_tag else "Unknown"
            has_stattrak = card.find("div", class_="bg-stattrak") is not None
            has_souvenir = card.find("div", class_="bg-souvenir") is not None

            formatted_skin_name = format_skin_name(weapon, skin_name)
            skin_data.append((formatted_skin_name, has_stattrak, has_souvenir))

        # Find next page link
        next_page = soup.find('a', rel="next")
        url = next_page['href'] if next_page else None
        time.sleep(random.uniform(2, 5))  # Random delay between requests
    
    return skin_data

def format_skin_name(weapon, skin_name):
    formatted_weapon = weapon_mapping.get(weapon, weapon)
    return f"{formatted_weapon} | {skin_name}" if skin_name != "Vanilla" else formatted_weapon

# Scraping all skins
all_skins = []
with tqdm(total=len(weapon_mapping), desc="Scraping Weapons", unit="weapon") as pbar:
    for weapon in weapon_mapping.keys():
        skins = get_skins_for_weapon(weapon)
        all_skins.extend(skins)
        pbar.update(1)

# Save structured skins to a file
with open("skins.txt", "w", encoding="utf-8") as file:
    for skin in all_skins:
        file.write(f"{repr(skin)}\n")

print(f"Extracted {len(all_skins)} skins. Saved to 'skins.txt'.")

Scraping Weapons: 100%|██████████| 62/62 [04:25<00:00,  4.29s/weapon]

Extracted 1665 skins. Saved to 'skins.txt'.


In [6]:
skin = get_skins_for_weapon("ak-47")
print(skin)

[('AK-47 | Inheritance', True, False), ('AK-47 | Slate', True, False), ('AK-47 | Ice Coaled', True, False), ('AK-47 | Nightwish', True, False), ('AK-47 | Legion of Anubis', True, False), ('AK-47 | The Outsiders', True, False), ('AK-47 | Asiimov', True, False), ('AK-47 | Head Shot', True, False), ('AK-47 | Elite Build', True, False), ('AK-47 | The Empress', True, False), ('AK-47 | Phantom Disruptor', True, False), ('AK-47 | Redline', True, False), ('AK-47 | Frontside Misty', True, False), ('AK-47 | Neon Revolution', True, False), ('AK-47 | Bloodsport', True, False), ('AK-47 | Aquamarine Revenge', True, False), ('AK-47 | Crossfade', False, False), ('AK-47 | Neon Rider', True, False), ('AK-47 | Point Disarray', True, False), ('AK-47 | Uncharted', True, False), ('AK-47 | Emerald Pinstripe', False, False), ('AK-47 | Cartel', True, False), ('AK-47 | Orbit Mk01', True, False), ('AK-47 | Vulcan', True, False), ('AK-47 | Safety Net', False, True), ('AK-47 | B the Monster', False, False), ('AK-4

In [7]:
print(len(all_skins))

1665


### Scrapping Case Skin

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

cases = [
    "Gallery Case", "Kilowatt Case", "Revolution Case", "Recoil Case", "Dreams & Nightmares-Case",
    "Chroma 2 Case", "Chroma 3 Case", "Chroma Case", "Clutch Case", "CS20 Case", "CS:GO Weapon Case", 
    "CS:GO Weapon Case 2", "CS:GO Weapon Case 3", "Danger Zone Case", "eSports 2013 Case", 
    "eSports 2013 Winter Case", "eSports 2014 Summer Case", "Falchion Case", "Fracture Case", "Gamma 2 Case", 
    "Gamma Case", "Glove Case", "Horizon Case", "Huntsman Case", "Operation Bravo Case", "Operation Breakout Weapon Case",
    "Operation Broken Fang Case", "Operation Hydra Case", "Operation Phoenix Weapon Case", "Operation Riptide Case",
    "Operation Vanguard Weapon Case", "Operation Wildfire Case", "Prisma 2 Case", "Prisma Case", "Revolver Case",
    "Shadow Case", "Shattered Web Case", "Snakebite Case", "Spectrum 2 Case", "Spectrum Case", "Winter Offensive Weapon Case"
]

base_item_url = "https://csgoskins.gg/items/"

case_urls = []

with tqdm(total=len(cases), desc="Scraping Cases", unit="case") as pbar:
    for case in cases:
        formatted_case_url = f"{case}"
        case_urls.append(formatted_case_url)
        pbar.update(1)  

with open("case.txt", "w", encoding="utf-8") as file:
    for url in case_urls:
        file.write(url + "\n")

print(f"✅ Extracted {len(case_urls)} case URLs. Saved to 'csgoskins_case_urls.txt'.")

Scraping Cases: 100%|██████████| 41/41 [00:00<?, ?case/s]

✅ Extracted 41 case URLs. Saved to 'csgoskins_case_urls.txt'.
